In [45]:
import pandas as pd
import unidecode
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

stemmer = SnowballStemmer('spanish')
tokenizer = RegexpTokenizer(r'\w+')

In [23]:
" ".join(["a", "b"])

'a b'

In [36]:
def process_tweet(sentence):
    sentence = sentence.lower()
    sentence = unidecode.unidecode(sentence)
    words = sentence.split()
    if len(words) > 1:
        if words[0].lower() == "rt" and "@" in words[1]:
            del words[1]
    sentence = " ".join(words)
    words = tokenizer.tokenize(sentence)
    words = [w for w in words if not w in stopwords.words("spanish")]
    words = [w for w in words if ("rt" != w.lower()) if (not "http" in w)]     # Delete words that contain mentions or the RT word 
    words = [stemmer.stem(w) for w in words] # Lemmatization spanish
    sentence = " ".join(words)
    return sentence

In [6]:
df_tweets= pd.read_csv("tweet_dataset.csv")

In [2]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)
# training the vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(["Hola qe tal"])

In [3]:
X_train

<1x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [7]:
Andalusia = ["Andalusia", "37.3399964,-4.5811614,250km"]
#Extremadura = 
Madrid = ["Madrid", "40.5248319,-3.7715628,60km"]
Catalonia = ["Catalonia", "41.8523094,1.5745043,150km"]
#CastillaLaMancha = 
#CastillaLeon = 
#Cantabria = 
Basque_Country = ["Basque_Country", "42.9911816,-2.5543023,100km"]
#ComunidadValenciana = 
Canary_Islands = ["Canary_Islands", "28.5306525,-15.7464439,400km"]
#Aragon = 
#LaRioja = 
#Navarra = 
#Asturias = 
#Murcia = 

CAS = [Andalusia, Madrid, Catalonia, Basque_Country, Canary_Islands]

In [38]:
if 200%100 == 0:
    print("yes")

yes


In [37]:
processed_dataset = []

n_rows = len(df_tweets["location"])
actual_row = 1
for CA in CAS:
    user_n = 1
    for index, row in df_tweets[df_tweets["location"] == CA[0]].iterrows(): 
        sentence = process_tweet(row[1])
        processed_dataset.append([row[0], sentence, row[2]])
        
        if actual_row %100 == 0:
            print("Row {} of {}".format(actual_row, n_rows))
        actual_row += 1
        

Row 2 of 27588
Row 3 of 27588
Row 4 of 27588
Row 5 of 27588
Row 6 of 27588
Row 7 of 27588
Row 8 of 27588
Row 9 of 27588
Row 10 of 27588
Row 11 of 27588
Row 12 of 27588
Row 13 of 27588
Row 14 of 27588
Row 15 of 27588
Row 16 of 27588
Row 17 of 27588
Row 18 of 27588
Row 19 of 27588
Row 20 of 27588
Row 21 of 27588
Row 22 of 27588
Row 23 of 27588
Row 24 of 27588
Row 25 of 27588
Row 26 of 27588
Row 27 of 27588
Row 28 of 27588
Row 29 of 27588
Row 30 of 27588
Row 31 of 27588
Row 32 of 27588
Row 33 of 27588
Row 34 of 27588
Row 35 of 27588
Row 36 of 27588
Row 37 of 27588
Row 38 of 27588
Row 39 of 27588
Row 40 of 27588
Row 41 of 27588
Row 42 of 27588
Row 43 of 27588
Row 44 of 27588
Row 45 of 27588
Row 46 of 27588
Row 47 of 27588
Row 48 of 27588
Row 49 of 27588
Row 50 of 27588
Row 51 of 27588
Row 52 of 27588
Row 53 of 27588
Row 54 of 27588
Row 55 of 27588
Row 56 of 27588
Row 57 of 27588
Row 58 of 27588
Row 59 of 27588
Row 60 of 27588
Row 61 of 27588
Row 62 of 27588
Row 63 of 27588
Row 64 of 27588


Row 497 of 27588
Row 498 of 27588
Row 499 of 27588
Row 500 of 27588
Row 501 of 27588
Row 502 of 27588
Row 503 of 27588
Row 504 of 27588
Row 505 of 27588
Row 506 of 27588
Row 507 of 27588
Row 508 of 27588
Row 509 of 27588
Row 510 of 27588
Row 511 of 27588
Row 512 of 27588
Row 513 of 27588
Row 514 of 27588
Row 515 of 27588
Row 516 of 27588
Row 517 of 27588
Row 518 of 27588
Row 519 of 27588
Row 520 of 27588
Row 521 of 27588
Row 522 of 27588
Row 523 of 27588
Row 524 of 27588
Row 525 of 27588
Row 526 of 27588
Row 527 of 27588
Row 528 of 27588
Row 529 of 27588
Row 530 of 27588
Row 531 of 27588
Row 532 of 27588
Row 533 of 27588
Row 534 of 27588
Row 535 of 27588
Row 536 of 27588
Row 537 of 27588
Row 538 of 27588
Row 539 of 27588
Row 540 of 27588
Row 541 of 27588
Row 542 of 27588
Row 543 of 27588
Row 544 of 27588
Row 545 of 27588
Row 546 of 27588
Row 547 of 27588
Row 548 of 27588
Row 549 of 27588
Row 550 of 27588
Row 551 of 27588
Row 552 of 27588
Row 553 of 27588
Row 554 of 27588
Row 555 of 275

Row 988 of 27588
Row 989 of 27588
Row 990 of 27588
Row 991 of 27588
Row 992 of 27588
Row 993 of 27588
Row 994 of 27588
Row 995 of 27588
Row 996 of 27588
Row 997 of 27588
Row 998 of 27588
Row 999 of 27588
Row 1000 of 27588
Row 1001 of 27588
Row 1002 of 27588
Row 1003 of 27588
Row 1004 of 27588
Row 1005 of 27588
Row 1006 of 27588
Row 1007 of 27588
Row 1008 of 27588
Row 1009 of 27588
Row 1010 of 27588
Row 1011 of 27588
Row 1012 of 27588
Row 1013 of 27588
Row 1014 of 27588
Row 1015 of 27588
Row 1016 of 27588
Row 1017 of 27588
Row 1018 of 27588
Row 1019 of 27588
Row 1020 of 27588
Row 1021 of 27588
Row 1022 of 27588
Row 1023 of 27588
Row 1024 of 27588
Row 1025 of 27588
Row 1026 of 27588
Row 1027 of 27588
Row 1028 of 27588
Row 1029 of 27588
Row 1030 of 27588
Row 1031 of 27588
Row 1032 of 27588
Row 1033 of 27588
Row 1034 of 27588
Row 1035 of 27588
Row 1036 of 27588
Row 1037 of 27588
Row 1038 of 27588
Row 1039 of 27588
Row 1040 of 27588
Row 1041 of 27588
Row 1042 of 27588
Row 1043 of 27588
Row 

Row 1461 of 27588
Row 1462 of 27588
Row 1463 of 27588
Row 1464 of 27588
Row 1465 of 27588
Row 1466 of 27588
Row 1467 of 27588
Row 1468 of 27588
Row 1469 of 27588
Row 1470 of 27588
Row 1471 of 27588
Row 1472 of 27588
Row 1473 of 27588
Row 1474 of 27588
Row 1475 of 27588
Row 1476 of 27588
Row 1477 of 27588
Row 1478 of 27588
Row 1479 of 27588
Row 1480 of 27588
Row 1481 of 27588
Row 1482 of 27588
Row 1483 of 27588
Row 1484 of 27588
Row 1485 of 27588
Row 1486 of 27588
Row 1487 of 27588
Row 1488 of 27588
Row 1489 of 27588
Row 1490 of 27588
Row 1491 of 27588
Row 1492 of 27588
Row 1493 of 27588
Row 1494 of 27588
Row 1495 of 27588
Row 1496 of 27588
Row 1497 of 27588
Row 1498 of 27588
Row 1499 of 27588
Row 1500 of 27588
Row 1501 of 27588
Row 1502 of 27588
Row 1503 of 27588
Row 1504 of 27588
Row 1505 of 27588
Row 1506 of 27588
Row 1507 of 27588
Row 1508 of 27588
Row 1509 of 27588
Row 1510 of 27588
Row 1511 of 27588
Row 1512 of 27588
Row 1513 of 27588
Row 1514 of 27588
Row 1515 of 27588
Row 1516 o

Row 1926 of 27588
Row 1927 of 27588
Row 1928 of 27588
Row 1929 of 27588
Row 1930 of 27588
Row 1931 of 27588
Row 1932 of 27588
Row 1933 of 27588
Row 1934 of 27588
Row 1935 of 27588
Row 1936 of 27588
Row 1937 of 27588
Row 1938 of 27588
Row 1939 of 27588
Row 1940 of 27588
Row 1941 of 27588
Row 1942 of 27588
Row 1943 of 27588
Row 1944 of 27588
Row 1945 of 27588
Row 1946 of 27588
Row 1947 of 27588
Row 1948 of 27588
Row 1949 of 27588
Row 1950 of 27588
Row 1951 of 27588
Row 1952 of 27588
Row 1953 of 27588
Row 1954 of 27588
Row 1955 of 27588
Row 1956 of 27588
Row 1957 of 27588
Row 1958 of 27588
Row 1959 of 27588
Row 1960 of 27588
Row 1961 of 27588
Row 1962 of 27588
Row 1963 of 27588
Row 1964 of 27588
Row 1965 of 27588
Row 1966 of 27588
Row 1967 of 27588
Row 1968 of 27588
Row 1969 of 27588
Row 1970 of 27588
Row 1971 of 27588
Row 1972 of 27588
Row 1973 of 27588
Row 1974 of 27588
Row 1975 of 27588
Row 1976 of 27588
Row 1977 of 27588
Row 1978 of 27588
Row 1979 of 27588
Row 1980 of 27588
Row 1981 o

...

Row 25613 of 27588
Row 25614 of 27588
Row 25615 of 27588
Row 25616 of 27588
Row 25617 of 27588
Row 25618 of 27588
Row 25619 of 27588
Row 25620 of 27588
Row 25621 of 27588
Row 25622 of 27588
Row 25623 of 27588
Row 25624 of 27588
Row 25625 of 27588
Row 25626 of 27588
Row 25627 of 27588
Row 25628 of 27588
Row 25629 of 27588
Row 25630 of 27588
Row 25631 of 27588
Row 25632 of 27588
Row 25633 of 27588
Row 25634 of 27588
Row 25635 of 27588
Row 25636 of 27588
Row 25637 of 27588
Row 25638 of 27588
Row 25639 of 27588
Row 25640 of 27588
Row 25641 of 27588
Row 25642 of 27588
Row 25643 of 27588
Row 25644 of 27588
Row 25645 of 27588
Row 25646 of 27588
Row 25647 of 27588
Row 25648 of 27588
Row 25649 of 27588
Row 25650 of 27588
Row 25651 of 27588
Row 25652 of 27588
Row 25653 of 27588
Row 25654 of 27588
Row 25655 of 27588
Row 25656 of 27588
Row 25657 of 27588
Row 25658 of 27588
Row 25659 of 27588
Row 25660 of 27588
Row 25661 of 27588
Row 25662 of 27588
Row 25663 of 27588
Row 25664 of 27588
Row 25665 of

Row 26078 of 27588
Row 26079 of 27588
Row 26080 of 27588
Row 26081 of 27588
Row 26082 of 27588
Row 26083 of 27588
Row 26084 of 27588
Row 26085 of 27588
Row 26086 of 27588
Row 26087 of 27588
Row 26088 of 27588
Row 26089 of 27588
Row 26090 of 27588
Row 26091 of 27588
Row 26092 of 27588
Row 26093 of 27588
Row 26094 of 27588
Row 26095 of 27588
Row 26096 of 27588
Row 26097 of 27588
Row 26098 of 27588
Row 26099 of 27588
Row 26100 of 27588
Row 26101 of 27588
Row 26102 of 27588
Row 26103 of 27588
Row 26104 of 27588
Row 26105 of 27588
Row 26106 of 27588
Row 26107 of 27588
Row 26108 of 27588
Row 26109 of 27588
Row 26110 of 27588
Row 26111 of 27588
Row 26112 of 27588
Row 26113 of 27588
Row 26114 of 27588
Row 26115 of 27588
Row 26116 of 27588
Row 26117 of 27588
Row 26118 of 27588
Row 26119 of 27588
Row 26120 of 27588
Row 26121 of 27588
Row 26122 of 27588
Row 26123 of 27588
Row 26124 of 27588
Row 26125 of 27588
Row 26126 of 27588
Row 26127 of 27588
Row 26128 of 27588
Row 26129 of 27588
Row 26130 of

Row 26530 of 27588
Row 26531 of 27588
Row 26532 of 27588
Row 26533 of 27588
Row 26534 of 27588
Row 26535 of 27588
Row 26536 of 27588
Row 26537 of 27588
Row 26538 of 27588
Row 26539 of 27588
Row 26540 of 27588
Row 26541 of 27588
Row 26542 of 27588
Row 26543 of 27588
Row 26544 of 27588
Row 26545 of 27588
Row 26546 of 27588
Row 26547 of 27588
Row 26548 of 27588
Row 26549 of 27588
Row 26550 of 27588
Row 26551 of 27588
Row 26552 of 27588
Row 26553 of 27588
Row 26554 of 27588
Row 26555 of 27588
Row 26556 of 27588
Row 26557 of 27588
Row 26558 of 27588
Row 26559 of 27588
Row 26560 of 27588
Row 26561 of 27588
Row 26562 of 27588
Row 26563 of 27588
Row 26564 of 27588
Row 26565 of 27588
Row 26566 of 27588
Row 26567 of 27588
Row 26568 of 27588
Row 26569 of 27588
Row 26570 of 27588
Row 26571 of 27588
Row 26572 of 27588
Row 26573 of 27588
Row 26574 of 27588
Row 26575 of 27588
Row 26576 of 27588
Row 26577 of 27588
Row 26578 of 27588
Row 26579 of 27588
Row 26580 of 27588
Row 26581 of 27588
Row 26582 of

Row 26971 of 27588
Row 26972 of 27588
Row 26973 of 27588
Row 26974 of 27588
Row 26975 of 27588
Row 26976 of 27588
Row 26977 of 27588
Row 26978 of 27588
Row 26979 of 27588
Row 26980 of 27588
Row 26981 of 27588
Row 26982 of 27588
Row 26983 of 27588
Row 26984 of 27588
Row 26985 of 27588
Row 26986 of 27588
Row 26987 of 27588
Row 26988 of 27588
Row 26989 of 27588
Row 26990 of 27588
Row 26991 of 27588
Row 26992 of 27588
Row 26993 of 27588
Row 26994 of 27588
Row 26995 of 27588
Row 26996 of 27588
Row 26997 of 27588
Row 26998 of 27588
Row 26999 of 27588
Row 27000 of 27588
Row 27001 of 27588
Row 27002 of 27588
Row 27003 of 27588
Row 27004 of 27588
Row 27005 of 27588
Row 27006 of 27588
Row 27007 of 27588
Row 27008 of 27588
Row 27009 of 27588
Row 27010 of 27588
Row 27011 of 27588
Row 27012 of 27588
Row 27013 of 27588
Row 27014 of 27588
Row 27015 of 27588
Row 27016 of 27588
Row 27017 of 27588
Row 27018 of 27588
Row 27019 of 27588
Row 27020 of 27588
Row 27021 of 27588
Row 27022 of 27588
Row 27023 of

Row 27431 of 27588
Row 27432 of 27588
Row 27433 of 27588
Row 27434 of 27588
Row 27435 of 27588
Row 27436 of 27588
Row 27437 of 27588
Row 27438 of 27588
Row 27439 of 27588
Row 27440 of 27588
Row 27441 of 27588
Row 27442 of 27588
Row 27443 of 27588
Row 27444 of 27588
Row 27445 of 27588
Row 27446 of 27588
Row 27447 of 27588
Row 27448 of 27588
Row 27449 of 27588
Row 27450 of 27588
Row 27451 of 27588
Row 27452 of 27588
Row 27453 of 27588
Row 27454 of 27588
Row 27455 of 27588
Row 27456 of 27588
Row 27457 of 27588
Row 27458 of 27588
Row 27459 of 27588
Row 27460 of 27588
Row 27461 of 27588
Row 27462 of 27588
Row 27463 of 27588
Row 27464 of 27588
Row 27465 of 27588
Row 27466 of 27588
Row 27467 of 27588
Row 27468 of 27588
Row 27469 of 27588
Row 27470 of 27588
Row 27471 of 27588
Row 27472 of 27588
Row 27473 of 27588
Row 27474 of 27588
Row 27475 of 27588
Row 27476 of 27588
Row 27477 of 27588
Row 27478 of 27588
Row 27479 of 27588
Row 27480 of 27588
Row 27481 of 27588
Row 27482 of 27588
Row 27483 of

In [39]:
df_processed = pd.DataFrame(processed_dataset, columns = ["id", "tweet", "location"])

In [41]:
df_processed.to_csv("tweet_dataset_processed.csv", encoding='utf-8-sig', index = False)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(df_processed['tweet'], df_processed['location'], test_size = 0.1, random_state = 1)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [50]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
prediction = model.predict(X_test)
accuracy = metrics.accuracy_score(prediction,y_test)

C:\Users\ignac\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [51]:
accuracy

0.48278361725262775